# llm 기본틀 만들기
- OpenAI Key 로딩, ChatOpenAI() 객체 생성

In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv(override=True)

api_key = os.getenv('OPENAI_API_KEY')
# print(api_key)

# 모델 객체 생성


In [2]:
# gpt model 객체 생성
# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001EC96407C50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EC962DD4F0>, root_client=<openai.OpenAI object at 0x000001EC93DB2240>, root_async_client=<openai.AsyncOpenAI object at 0x000001EC96407DA0>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 시스템 프롬프트 작성


Want assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?".

In [33]:
system_prompt = """
Want assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?".
"""
user_prompt = """
{company}에 주식을 투자해도 될까요? 마크다운 형식의 투자보고서를 한글로 작성해 주세요.
아래의 기본정보, 재무제표를 검토해서 마크다운 형식의 투자보고서를 한글로 작성해 주세요.

기본정보 :
{basic_info}

재무제표 :
{finacial_statement}

"""

# 프롬프트 객체 생성


In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])
prompt

ChatPromptTemplate(input_variables=['basic_info', 'company', 'finacial_statement'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nWant assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?".\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['basic_info', 'company', 'finacial_statement'], input_types={}, partial_variables={}, template='\n{company}에 주식을 투자해도 될까요? 마크다운 형식의 투자보고서를 한글로 작성해 주세요.\n아래의 기본정보, 

# OutPut 파서 객체 생성


In [35]:
output_parser = StrOutputParser()
output_parser

StrOutputParser()

# LCEL로 LLM 요청 및 응답받기
- chain 구성 : LCEL(LangChain Expression Language)



In [36]:
# from stock_info import Stock 인식이 안될 경우 아래 코드 실행
import sys
import os

# 상위 디렉터리(LANGCHAIN_PJT)를 경로에 추가
sys.path.append(os.path.abspath(".."))
import stock_info

In [37]:
from stock_info import Stock

# 회사 이름
company = "MicroSoft"
symbol = "MSFT"


In [38]:
# chain 구성 : LCEL(LangChain Expression Language)
chain = prompt | llm | output_parser

# 회사 이름
# company = "MicroSoft"
# symbol = "MSFT"  # stock의 symbol 정보

company = "Apple Inc"
symbol = "AAPL"
# 기본정보 :  basic_info
# 재무제표: finacial_statement

# stock 정보를 객체로 모듈화해서 불러오기
from stock_info import Stock

stock = Stock(symbol)
req_value = {
    "company":company,
        # 기본정보 :  basic_info
    "basic_info": stock.get_basic_info(),
    # 재무제표: finacial_statement
    "finacial_statement" : stock.get_financial_statement()
            }

response = chain.invoke(req_value)

In [39]:
# print(company)
# print(stock.get_basic_info())
# print(stock.get_financial_statement())
print(response)


## 투자보고서: Apple Inc. (AAPL)

### 기본정보
- 기업명: Apple Inc.
- 시가총액: 2,968,938,348,544
- 산업: 소비자 전자제품
- 섹터: 기술
- 정규직 종업원 수: 164,000 명
- 현재 주가: 198.78
- 기업가치: 3,076,731,961,344

### 재무제표

#### 손익계산서
| 항목              | 2024년 | 2023년 | 2022년 | 2021년 |
|-------------------|---------|---------|---------|---------|
| 총 수익           | 3.91035e+11 | 3.83285e+11 | 3.94328e+11 | 3.65817e+11 |
| 총 이익           | 1.80683e+11 | 1.69148e+11 | 1.70782e+11 | 1.52836e+11 |
| 영업 이익         | 1.23216e+11 | 1.14301e+11 | 1.19437e+11 | 1.08949e+11 |
| 순 이익           | 9.3736e+10 | 9.6995e+10 | 9.9803e+10 | 9.468e+10 |

#### 대차대조표
| 항목                                   | 2024년 | 2023년 | 2022년 | 2021년 |
|---------------------------------------|---------|---------|---------|---------|
| 총 자산                               | 3.6498e+11 | 3.52583e+11 | 3.52755e+11 | 3.51002e+11 |
| 총 부채 및 소수 지분 순액   | 3.0803e+11 | 2.90437e+11 | 3.02083e+11 | 2.87912e+11 |
| 주주자본                             | 5.695e+10 | 6.2146e+10 |